In [56]:
import ee
import geopandas as gpd
import fiona
import shapely
import contextily as cx
import json
from src.visualization import ee_viz
from src.data import mtbs
from src.data import shape_processor
import src.data.ee_utils as ee_utils
import numpy as np

import importlib
importlib.reload(ee_viz)
importlib.reload(ee_utils)
importlib.reload(mtbs)

<module 'src.data.mtbs' from '/home/jk871/fire-regen/src/data/mtbs.py'>

In [42]:
ee.Initialize()

### Get Regions of interest

In [43]:
# Fetch simplified regions of interest - to avoid processing problems with really complicated shapes.
seki = gpd.read_file("../../data/shapefiles/seki_convex_hull.shp")
sierras = gpd.read_file("../../data/shapefiles/sierras_convex_hull.shp")

In [44]:
seki_complex = gpd.read_file("../../data/shapefiles/seki.shp")
seki_box = shape_processor.get_box(seki_complex)

In [ ]:
seki.to_crs(3857).explore()

In [ ]:
sierras.to_crs(3857).explore()

### Get Burn Severity Data

In [45]:
burn_severity_ic = mtbs.get_burn_severity_data().sort('system:time_start', False)

In [48]:
ee_viz.viz_burn_severity(burn_severity_ic.first(), seki.geometry.iloc[0], ['burn_severity', 'burn_year'])

Map(center=[36.72976974493908, -118.62809181119908], controls=(WidgetControl(options=['position', 'transparent…

### Get aggregated burn data - last burn severity for each pixel, and burn count for each pixel

In [57]:
burn_data = mtbs.get_aggregated_burn_data()
burn_data

In [58]:
ee_viz.viz_burn_severity(burn_data, seki.geometry.iloc[0], ['last_burn_severity', 'last_burn_year', 'burn_count'])

Map(center=[36.72976974493908, -118.62809181119908], controls=(WidgetControl(options=['position', 'transparent…

### Save Fire Raster as Geo Tiff

In [59]:
ee_utils.save_image_to_drive(burn_data, polygon=seki_box.geometry[0], img_name='burn_data', scale=30)

Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id: F7OXCQRMLF3GF7EXAJV7TMAH).
Polling for task (id

In [ ]:
# Import Group project shapes of interests
seki = gpd.read_file("../../data/shapefiles/SEKI_outline.shp")
seki.explore()

In [ ]:
print(f'SEKI area in km^2: {(seki.geometry.area/10**6).sum()}')

In [ ]:
layers = fiona.listlayers('../../data/fire_perimeters.gdb/')
layers

First layer, 'firep21_2' is fire perimeters.

Second layer, 'rxburn21_2' are prescribed burns.

Third layer, 'Non_RXFire_Legacy13_2' seem to be other treatment types.

In [ ]:
gdf_firep = gpd.read_file('../../data/fire_perimeters.gdb/', layer=layers[0])

In [ ]:
firep = gdf_firep.to_crs(3857)

In [ ]:
gdf_firep

In [ ]:
seki_union = gpd.GeoDataFrame({'geometry': gpd.GeoSeries([shapely.unary_union(seki.geometry)])})
seki_union

In [ ]:
seki_fires = firep.sjoin(seki_union, how="inner")

In [ ]:
seki_fires

In [ ]:
seki_fires_since_2000 = seki_fires[seki_fires.ALARM_DATE > '2000-01-01'].sort_values('ALARM_DATE')

In [ ]:
seki_fires_since_2000

In [ ]:
seki_fires_since_2000.to_file("../../data/shapefiles/gee/seki_fires_since_2000.shp")

In [ ]:
ax = firep[firep.index==21424].overlay(seki_union, how="union").plot(figsize=(20,10), cmap='RdYlBu', alpha=0.8)
cx.add_basemap(ax)

In [ ]:
newdf = firep.sjoin(seki, how="inner").overlay(seki, how="symmetric_difference")

In [ ]:
newdf.plot()

### Create EE Rasters for fires

In [ ]:
image = ee.Image.constant(1).rename('isfire')

In [ ]:
fire = firep[firep.index==237]

In [ ]:
import numpy as np
def gdf_to_ee_polygon(gdf_polygon: shapely.Polygon):
    ''' Helper to convert GeoPandas geometry to Earth Engine geometry. '''
    x, y = gdf_polygon.exterior.coords.xy
    coords = np.dstack((x, y)).tolist()
    return ee.Geometry.Polygon(coords)

In [ ]:
len(list(fire.geometry.iloc[0].geoms))

In [ ]:
list(fire.geometry.iloc[0].geoms)

In [ ]:
fire.overlay(gpd.GeoDataFrame({'geometry': gpd.GeoSeries([list(fire.geometry.iloc[0].geoms)[12]])}), how='symmetric_difference').plot(cmap='tab20b')

In [ ]:
gpd.GeoSeries([list(fire.geometry.iloc[0].geoms)[17]]).plot()

In [ ]:
list(fire.geometry.iloc[0].geoms)[17].bounds

In [ ]:
ee_polygons = []
count = 0
for polygon in list(fire.geometry.iloc[0].geoms):
    ee_polygons.append(gdf_to_ee_polygon(polygon))
    count += 1
    #if count > 45:
    #    break
multi = ee.Geometry.MultiPolygon(ee_polygons)

In [ ]:
multi.getInfo()

In [ ]:
from shapely.ops import cascaded_union
fire_polygon = shapely.unary_union(list(fire.geometry.iloc[0].geoms))

In [ ]:
type(fire_polygon)

In [ ]:
len(list(fire_polygon.geoms))

In [ ]:
fire_json = fire.to_json()

In [ ]:
featureCollection = ee.FeatureCollection(json.loads(fire_json))

In [ ]:
coords = list(fire.geometry.iloc[0].geoms)[17].bounds
box_poly = shapely.box(coords[0], coords[1], coords[2], coords[3])

In [ ]:
#clipped = image.clip(gdf_to_ee_polygon(shapely.Polygon(list(fire.geometry.iloc[0].geoms)[17].bounds)))
clipped = image.clip(gdf_to_ee_polygon(box_poly))

In [ ]:
ee_viz.viz_image(clipped, polygon=fire.geometry.iloc[0], bands=["isfire"], band_names=['isfire'])

In [ ]:
import geemap
map = geemap.Map(zoom=9)

In [ ]:
map.addLayer(image)

In [ ]:
map.display()

In [ ]:
import geemap
import ee
ee.Initialize()
# Create a default map
Map = geemap.Map()

# Load an image.
#image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {
  'bands': ['isfire'],
  'min': 0,
  'max': 2
}

# Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 8) # San Francisco Bay
Map.addLayer(clipped, vizParams, 'false color composite')

# Display the map
Map

In [ ]:
Map